Notebook to convert masks between different formats: png/tif, rle, json.

In [ ]:
import json
import tifffile
import os
import numpy as np
import pandas as pd
import codecs
import cv2
import matplotlib.pyplot as plt
import shutil
import skimage.io as io
from PIL import Image


JSON mask to image mask (png/tif)

In [ ]:
def get_mask_from_json(jsn:json,shape=(3000 , 3000)):
    
    #code from colon deepflash5
    
    data = json.load(codecs.open(jsn, 'r', 'utf-8-sig'))
#json.load(open(jsn).read().decode('utf-8-sig'))
    polys = []
    for index in range(data.__len__()):
        
        
        
        geom = [np.array([x]) for x in data[index]['geometry']['coordinates']]
        
        for g in geom:
            polys.append(g.astype(int))
        
        
            
        
    mask = np.zeros(shape)
    for i in range(len(polys)):
        if cv2.contourArea(polys[i].astype(int)) >= 10:
            cv2.fillPoly(mask, polys[i], 1)
        
    return mask

In [ ]:
filename = "filename"
img_shape = tifffile.imread(f"{filename}.tif").shape
print(f"Shape: {img_shape}")
generated_mask =  np.uint8(get_mask_from_json(f"{filename}.json",(img_shape[0],img_shape[1]))*255)
plt.imshow(generated_mask)
io.imsave(f"{filename}.png",generated_mask)  

# 2 ways to save as .tif
#io.imsave(f"{filename}.tif",generated_mask)  # to save as .tif
#tifffile.imwrite(f"{filename}.tif",generated_mask) # to save as .tif

RLE to png mask

In [ ]:
def rle2mask(rle, shape):
    '''
    mask_rle: run-length as string formatted (start length)
    shape: (height, width) of array to return 
    Returns numpy array <- 1(mask), 0(background)
    '''
    s = rle.split()
    starts, lengths = [np.asarray(x, dtype=int) for x in (s[0:][::2], s[1:][::2])]
    starts -= 1
    ends = starts + lengths
    img = np.zeros(shape[0]*shape[1], dtype=np.uint8)
    for lo, hi in zip(starts, ends):
        img[lo:hi] = 1
    return img.reshape(shape, order='F')


result_csv = pd.read_csv("csv_file_with_imagename_and_rle.csv")

#reading image name and rle from the csv
image_index_in_csv = 10

im_file_name,rle = result_csv.iloc[image_index_in_csv,0],result_csv.iloc[image_index_in_csv,1]

#deriving image shape for the rle
img_shape = tifffile.imread("path_to_image_file.tif").shape

#rle to mask conversion
mask = rle2mask(rle,(img_shape[0],img_shape[1]))

#comment this if not required
plt.imshow(mask)
plt.show()

#saving masK as png
Image.fromarray(mask*255).save("mask.png")

image_mask.tif to RLE. Requires the conversion of JSON masks to .tif format first.

In [ ]:
def rle_encode(img):
    #cite = https://www.kaggle.com/lifa08/run-length-encode-and-decode
    '''
    img: numpy array, 1 - mask, 0 - background
    Returns run length as string formated
    '''
    pixels = img.T.flatten()
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    return ' '.join(str(x) for x in runs)

rle = rle_encode(tifffile.imread("path_to_mask_image.tif"))

Png Image mask to JSON + Generate mask overlay + Get instance counts and areas.

In [ ]:
def json_generation_from_mask(mask_path:str,json_path:str,image_df,overlay_dir):
    '''
    mask_path: path to png mask file
    json_path: path to save json
    image_df: dataframe with image filenames
    overlay_dir: path to save overlays
    Returns count, areas of instances in mask
    '''
    areas = []
    annot = []
    mask_image_name = (mask_path).split("/")[-1]
    msk = cv2.imread(mask_path,0)
    # print( mask_image_name.replace(".png","").replace("mask_",""))
    image_path = [x for x in image_df['id'] if mask_image_name.replace(".png","").replace("mask_","") in x]
    image = tifffile.imread(image_path[0])
    
    font = cv2.FONT_HERSHEY_SIMPLEX
  
    # fontScale
    fontScale = 0.5

    # Blue color in BGR
    color = (0, 0, 255)

    # Line thickness of 2 px
    thickness = 1
    contours, hierarchy = cv2.findContours(msk.astype(np.uint8), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    
#     data_pred_overlay = cv2.drawContours(image.copy(), contours, -1, (0, 255, 0), 3)
#     cv2.imwrite(f"{overlay_dir}/pred_overlays_"+mask_image_name.replace("mask_","").replace(".png",".tif"),data_pred_overlay)
    data_pred_overlay = image.copy()
    data = {}
    if len(contours)==0:
        data =  {'type':'Feature','geometry':{'type':'Polygon','coordinates':[]}}
        print(f"No annotations for {mask_image_name}")
    else:
        for i,c in enumerate(contours):
            # print(c.shape)
            area = cv2.contourArea(c)
            if area < 2000:
                continue
            areas.append(area)
            data_pred_overlay = cv2.drawContours(data_pred_overlay, c, -1, (0, 255, 0), 3)
            
            data_pred_overlay  = cv2.putText(data_pred_overlay, str(area), (c[0][0][0],c[0][0][1]), font, 
                   fontScale, color, thickness, cv2.LINE_AA)
            
            data = {'type':'Feature','geometry':{'type':'Polygon','coordinates':[[x[0] for x in c.tolist()]+[c.tolist()[0][0]]]}}
            annot.append(data)
    tifffile.imwrite(f"{overlay_dir}/pred_overlays_"+mask_image_name.replace("mask_","").replace(".png",".tif"),data_pred_overlay)
    with open(json_path+mask_image_name.replace("mask_","").replace(".png","_annotations.json"), 'w') as outfile:
        json.dump(annot, outfile)
        
    return len(contours),areas